In [1]:
import socket
import numpy as np
import cv2 as cv
import threading

skt = socket.socket()
skt.bind(("192.168.43.49", 1234))
 #to continue connectivity
skt.listen()
#accepting request from any server
session, address = skt.accept() # session to memorise or store information
print(session.recv(2046)) 
# using camera from IPCamera App
cameraIndex = '192.168.43.1:8080/video' 
camera = cv.VideoCapture(cameraIndex) # to start the camera 

def sender():
    while True:
        status, photo = camera.read()
        photo = cv.resize(photo, (640, 480))
        print(photo.shape)
        if status:
            session.send(np.ndarray.tobytes(photo))
        else: print( " Frame is not seen " )

def receiver():
    framesLost = 0
    print("Entered")
    while True:
        framesLost += 1
        data = session.recv(100000000)
        if(data == b'finished'):
            
            print("Finished")
            camera.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv.imshow('From Client', photo.reshape(480, 640, 3))
                
                if cv.waitKey(1) == 13:
                    session.send(b'finished')
                    camera.release()
                    cv.destroyAllWindows()
                    break
            else:
                print("Lost {} frames".format(framesLost) )
                
                        
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

In [ ]:
http://192.168.43.1:8080/video